In [36]:
import requests
from bs4 import BeautifulSoup

import pandas as pd

In [ ]:
ACCESS_TOKEN = ''

In [42]:
class GeniusLM:
  
    def __init__(self, token):
        self.token = token
    
    def search(self, value):
        page_number = 1
        songs = []
        while True:
            genius_search_url = f"http://api.genius.com/search?q={value}&access_token={self.token}&page={page_number}"
            response = requests.get(genius_search_url)
            hits = response.json()['response']['hits']
            if hits:
                for song in hits:
                    songs.append(GeniusSongLM(song["result"]["id"],
                                              song["result"]["title"],
                                              song["result"]["url"],
                                              song["result"]["path"],
                                              song["result"]["header_image_url"],
                                              song["result"]["annotation_count"],
                                              song["result"]["pyongs_count"],
                                              song["result"]["primary_artist"]["id"],
                                              song["result"]["primary_artist"]["name"],
                                              song["result"]["primary_artist"]["url"],
                                              song["result"]["primary_artist"]["image_url"]
                                             ))
                page_number = page_number + 1
            else:
                break
        return songs
    
    def get_dataframe(self, songs):
        return pd.DataFrame([s.__dict__ for s in songs])
    
class GeniusSongLM:
  
    def __init__(self, id_song, title, url, path, header_image_url, annotation_count, pyongs_count, primary_artist_id, primary_artits_name, primary_artits_url, primary_artits_imageurl):
        self.id_song = id_song
        self.title = title 
        self.url = url
        self.path = path
        self.song = self._get_song(self.path)
        self.header_image_url = header_image_url,  
        self.annotation_count = annotation_count 
        self.pyongs_count = pyongs_count
        self.primary_artist_id = primary_artist_id 
        self.primary_artits_name = primary_artits_name
        self.primary_artits_url = primary_artits_url
        self.primary_artits_imageurl = primary_artits_imageurl
        
    def _get_song(self, path):
        url = "https://genius.com" + path
        response_song = requests.get(url)
        html = BeautifulSoup(response_song.content, "lxml")
        lyrics = html.select('div[class^="Lyrics__Container"], .song_body-lyrics p')
        song = ''
        for div in lyrics:
            song = song + ' '
            song = song + div.text
        return song

In [41]:
genius = GeniusLM(ACCESS_TOKEN)
songs = genius.search('lisboa')
dataframe = genius.get_dataframe(songs)
dataframe.head()

,id_song,title,url,path,song,header_image_url,annotation_count,pyongs_count,primary_artist_id,primary_artits_name,primary_artits_url,primary_artits_imageurl
0,3773805,Nova Lisboa,https://genius.com/Dino-dsantiago-nova-lisboa-...,/Dino-dsantiago-nova-lisboa-lyrics,"[Letra de ""Nova Lisboa""][Verso 1]Qual é a ide...",(https://images.genius.com/b0a6a41bf9b22cc5d9a...,0,1.0,557991,Dino d’Santiago,https://genius.com/artists/Dino-dsantiago,https://images.genius.com/f01fa4552a486d751c5b...
1,2933019,Gangs de Lisboa,https://genius.com/Allen-halloween-gangs-de-li...,/Allen-halloween-gangs-de-lisboa-lyrics,"[Letra de ""Gangs de Lisboa""][Intro: Allen Hal...",(https://images.genius.com/5a95662724890e4c44b...,0,1.0,60345,Allen Halloween,https://genius.com/artists/Allen-halloween,https://images.genius.com/9510db21b25d06ba906e...
2,860391,Cheira a Lisboa,https://genius.com/Amalia-rodrigues-cheira-a-l...,/Amalia-rodrigues-cheira-a-lisboa-lyrics,"[Letra de ""Cheira a Lisboa""][Estrofe 1]Lisboa...",(https://images.genius.com/b1548fb744592253e95...,1,NaN,42271,Amália Rodrigues,https://genius.com/artists/Amalia-rodrigues,https://images.genius.com/4bc43282ff18dcf594fd...
3,6329407,Lisboa,https://genius.com/Anavitoria-and-lenine-lisbo...,/Anavitoria-and-lenine-lisboa-lyrics,"[Letra de ""Lisboa"" com ANAVITÓRIA feat. Lenin...",(https://images.genius.com/3d8b71c9df94554426f...,0,1.0,2545970,Anavitória & Lenine,https://genius.com/artists/Anavitoria-and-lenine,https://assets.genius.com/images/default_avata...
4,6855047,Lisboa-Madrid,https://genius.com/Anavitoria-and-jorge-drexle...,/Anavitoria-and-jorge-drexler-lisboa-madrid-ly...,"[Letra de ""Lisboa-Madrid"" com Anavitória & Jo...",(https://images.genius.com/ec90c545be281bd891b...,0,NaN,2751573,Anavitória & Jorge Drexler,https://genius.com/artists/Anavitoria-and-jorg...,https://assets.genius.com/images/default_avata...
